# Классификация и сегментация клиентов iFood





### Постановка задачи



### Компания iFood собирает данные о клиентах и их покупках. Цель — понять поведение клиентов, выделить сегменты (кластеризация) и научиться предсказывать, откликнется ли клиент на маркетинговую кампанию (классификация, целевая переменная - Response).



### Подключение модулей

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.decomposition import PCA
%matplotlib inline
import warnings
warnings.simplefilter(action='ignore', category=[FutureWarning, UserWarning])
sns.set_style("whitegrid")

In [ ]:
df = pd.read_csv("ifood_df.csv")

In [ ]:
df.head()

# Описание данных
# Числовые признаки:

### Income — годовой доход
### Kidhome, Teenhome — количество детей и подростков
### Recency — дней с последней покупки
### MntWines, MntFruits, MntMeatProducts, ... — суммы трат по категориям
### NumDealsPurchases, NumWebPurchases, ... — количество покупок разными каналами
### NumWebVisitsMonth — посещений сайта за месяц
### AcceptedCmp1–5, AcceptedCmpOverall — принятие предыдущих кампаний
### Age — возраст
### Customer_Days — сколько дней клиент
### MntTotal, MntRegularProds — итоговые траты

# Целевая переменная:

### Response — отклик на последнюю кампанию (0 / 1)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe().round(2)

Распределение целевой переменной

In [ ]:
plt.figure(figsize=(5, 4))
sns.countplot(x='Response',data=df,palette='viridis',hue='Response',legend=False)
plt.title('Распределение отклика на кампанию (Response)')
plt.xlabel('Response (0 — нет, 1 — да)')
plt.ylabel('Количество клиентов')
plt.show()

Класс сильно несбалансирован — около 85% клиентов не откликнулись

# Разведочный анализ и визуализация

### Корреляционная тепловая карта

In [ ]:
plt.figure(figsize=(14,10))
corr = df.corr(numeric_only=True)
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, cmap='RdBu_r', linewidths=0.4, center=0)
plt.title('Матрица корреляций')
plt.show()

Самые сильные связи - между доходом и общими тратами, а также между тратами на вино и общими тратами

### Распределения ключевых признаков

In [ ]:
key_cols = ['Income', 'MntTotal', 'Age', 'Recency', 'NumWebVisitsMonth']

fig, axes = plt.subplots(2, 3, figsize=(15,8))
axes = axes.ravel()

for i, col in enumerate(key_cols):
    sns.histplot(df[col], kde=True, ax=axes[i], color='teal')
    axes[i].set_title(f'Распределение {col}')

plt.tight_layout()
plt.show()

### Распределения ключевых признаков

In [ ]:
key_cols = ['Income', 'MntTotal', 'Age', 'Recency', 'NumWebVisitsMonth']

fig, axes = plt.subplots(2, 3, figsize=(15,8))
axes = axes.ravel()

for i, col in enumerate(key_cols):
    sns.histplot(df[col], kde=True, ax=axes[i], color='teal')
    axes[i].set_title(f'Распределение {col}')

plt.tight_layout()
plt.show()

Доход и общие траты имеют сильный правый хвост — большинство клиентов со средним и ниже среднего уровнем

### Затраты по категориям в зависимости от отклика

In [ ]:
import warnings
import matplotlib as mpl
warnings.filterwarnings("ignore", message=".*vert.*deprecated.*")
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="matplotlib")

In [ ]:
plt.figure(figsize=(12, 6))

sns.boxplot(x='Категория', y='Сумма', hue='Response', data=df_melt, palette='Set2')

plt.yscale('log')
plt.xticks(rotation=45)
plt.title('Затраты по категориям — отклик vs неотклик')
plt.legend(title='Отклик (0 — нет, 1 — да)')
plt.tight_layout()
plt.show()

Клиенты, откликнувшиеся на кампанию, значительно больше тратят на вино и мясные продукты

обновил